Setup


In [1]:
import logging

# define a formatter to display the messages to console (standard output)
console_formatter = logging.Formatter('%(message)s')
# console_formatter = logging.Formatter('%(levelname)s:%(module)s:%(message)s')
console_handler = logging.StreamHandler()
console_handler.setFormatter(console_formatter)
# define a logger for this notebook and attach the console handler
logger = logging.getLogger('Label-Propagation')
logger.handlers.clear()
logger.propagate = False
logger.addHandler(console_handler)
# set an appropriate level of logging for this notebook
logger.setLevel(logging.INFO)


Create data loader

In [2]:
# import os
# import cv2
# import albumentations
# import numpy as np
# import pandas as pd
# import pickle
# from tqdm.notebook import tqdm
# 
# class DataLoader:
#     """
#     DataLoader class for loading, processing, and transforming image, mask, and tabular data.
# 
#     Attributes:
#     -----------
#     data_folder : str
#         Path to the folder containing the dataset.
#     seg_folder : str
#         Path to the folder containing segmented image data.
#     tabular_folder : str
#         Path to the folder containing tabular annotation data.
#     transform : callable, optional
#         Transformation function for images and masks. By default, None.
#     dataset : dict
#         Dictionary holding images, masks, and corresponding annotations.
#     img_names : list
#         List of image file names or unique identifiers in the dataset.
#     """
# 
#     def __init__(
#         self,
#         data_folder,
#         transform=None
#     ):
#         """
#         Initializes the DataLoader class by loading the dataset from the given folder path,
#         including both image-mask pairs and associated tabular annotations.
# 
#         Parameters:
#         -----------
#         data_folder : str
#             Path to the dataset's main folder.
#         transform : callable, optional
#             Transformation function for images and masks. Defaults to None.
#         """
#         self.data_folder = data_folder
#         self.seg_folder = os.path.join(data_folder, "labelled_data/segments")
#         self.tabular_folder = os.path.join(data_folder, "tabular-data")
#         self.transform = transform
# 
#         # Load segmented image data from a pickle file
#         seg_ds_path = os.path.join(self.data_folder, "labelled_data/labelled_segmentation_data.pickle")
#         images, masks, img_paths = pickle.load(open(seg_ds_path, 'rb'))
#         print(f'Loaded {len(images)} images!')
# 
#         # Create a dataset dictionary with image paths as keys and image, mask data
#         self.dataset = {img_p: {'image': img, 'mask': msk} for img_p, img, msk in zip(img_paths, images, masks)}
#         self.img_names = list(self.dataset.keys())
# 
#         # Load tabular annotation data and add to the dataset
#         for annotation_df_path in tqdm(os.listdir(self.tabular_folder)):
#             if not annotation_df_path.startswith('annotation'):
#                 continue
# 
#             annotation_df = pd.read_csv(os.path.join(self.tabular_folder, annotation_df_path))
#             annotation_df.quadratid = annotation_df.quadratid.astype(str)
# 
#             # Match tabular data with image data in the dataset
#             for quadratid in annotation_df['quadratid'].unique():
#                 if quadratid not in self.dataset:
#                     continue
# 
#                 # Ensure no duplicate quadratid with different data exists
#                 if 'points' in self.dataset[quadratid]:
#                     raise Exception("Same quadrat, different datasets!")
# 
#                 # Add points and point labels to the dataset
#                 self.dataset[quadratid]['points'] = annotation_df[annotation_df['quadratid'] == quadratid][['x', 'y']].values
#                 self.dataset[quadratid]['point_labels'] = annotation_df[annotation_df['quadratid'] == quadratid]['func_group'].values
# 
#     def __len__(self):
#         """
#         Returns the total number of samples in the dataset.
# 
#         Returns:
#         --------
#         int
#             Number of samples in the dataset.
#         """
#         return len(self.dataset)
# 
#     def __getitem__(self, idx):
#         """
#         Returns the image, mask, keypoints, and point labels for the given index.
# 
#         Parameters:
#         -----------
#         idx : int
#             Index of the sample to retrieve.
# 
#         Returns:
#         --------
#         tuple
#             A tuple containing the image, mask, keypoints, and point labels.
#         """
#         img_dict = self.dataset[self.img_names[idx]]
#         img, mask, keypoints, point_labels = img_dict['image'], img_dict['mask'], img_dict['points'], img_dict['point_labels']
# 
#         # Apply transformation if specified
#         if self.transform:
#             transform = self.get_resize_transform(image_size=1024)
#             transformed = transform(image=img, mask=mask)
#             img, mask = transformed['image'], transformed['mask']
# 
#         return img, mask, keypoints, point_labels
# 
#     @staticmethod
#     def get_resize_transform(image_size=1024):
#         """
#         Creates a resizing transformation for images and masks to a specified size.
# 
#         Parameters:
#         -----------
#         image_size : int, optional
#             Desired output size of the image and mask. Defaults to 1024.
# 
#         Returns:
#         --------
#         albumentations.Compose
#             A composed transformation that resizes images and masks.
#         """
#         resize_transform = albumentations.Compose(
#             [
#                 albumentations.Resize(
#                     height=image_size,
#                     width=image_size,
#                     interpolation=cv2.INTER_AREA,
#                     p=1,
#                 ),
#             ]
#         )
#         return resize_transform


In [3]:
# import logging
# import os
# import pickle
# import pandas as pd
# from tqdm.notebook import tqdm
# import cv2
# import albumentations
# import numpy as np
# 
# # Define a formatter to display messages to the console (standard output)
# console_formatter = logging.Formatter('%(message)s')
# console_handler = logging.StreamHandler()
# console_handler.setFormatter(console_formatter)
# logger = logging.getLogger('Label-Propagation')
# logger.handlers.clear()
# logger.propagate = False
# logger.addHandler(console_handler)
# logger.setLevel(logging.INFO)
# 
# class DataLoader:
#     """
#     DataLoader class for loading, processing, and transforming image, mask, and tabular data.
#     """
# 
#     def __init__(self, data_folder, transform=None):
#         self.data_folder = data_folder
#         self.seg_folder = os.path.join(data_folder, "labelled_data")
#         self.tabular_folder = data_folder  # Use the same folder for tabular data
#         self.transform = transform
# 
#         # Load segmented image data from a pickle file
#         seg_ds_path = os.path.join(self.seg_folder, "labelled_segmentation_data.pickle")
#         logger.info(f"Trying to load segmentation data from: {seg_ds_path}")  # Debug print
# 
#         # Check if the file exists
#         if not os.path.exists(seg_ds_path):
#             logger.error(f"File does not exist: {seg_ds_path}")
#             raise FileNotFoundError(f"The segmentation data file was not found: {seg_ds_path}")
# 
#         images, masks, img_paths = pickle.load(open(seg_ds_path, 'rb'))
#         logger.info(f'Loaded {len(images)} images!')
# 
#         # Create a dataset dictionary with image paths as keys and image, mask data
#         self.dataset = {img_p: {'image': img, 'mask': msk} for img_p, img, msk in zip(img_paths, images, masks)}
#         self.img_names = list(self.dataset.keys())
# 
#         # Load tabular annotation data and add to the dataset
#         for annotation_df_path in tqdm(os.listdir(self.tabular_folder)):
#             if not annotation_df_path.startswith('annotation'):
#                 continue
# 
#             annotation_df = pd.read_csv(os.path.join(self.tabular_folder, annotation_df_path))
#             annotation_df.quadratid = annotation_df.quadratid.astype(str)
# 
#             # Match tabular data with image data in the dataset
#             for quadratid in annotation_df['quadratid'].unique():
#                 if quadratid not in self.dataset:
#                     continue
# 
#                 # Ensure no duplicate quadratid with different data exists
#                 if 'points' in self.dataset[quadratid]:
#                     raise Exception("Same quadrat, different datasets!")
# 
#                 # Add points and point labels to the dataset
#                 self.dataset[quadratid]['points'] = annotation_df[annotation_df['quadratid'] == quadratid][['x', 'y']].values
#                 self.dataset[quadratid]['point_labels'] = annotation_df[annotation_df['quadratid'] == quadratid]['func_group'].values
# 
#     def __len__(self):
#         return len(self.dataset)
# 
#     def __getitem__(self, idx):
#         img_dict = self.dataset[self.img_names[idx]]
#         img, mask, keypoints, point_labels = img_dict['image'], img_dict['mask'], img_dict['points'], img_dict['point_labels']
# 
#         if self.transform:
#             transformed = self.transform(image=img, mask=mask)
#             img, mask = transformed['image'], transformed['mask']
# 
#         return img, mask, keypoints, point_labels
# 
#     @staticmethod
#     def get_resize_transform(image_size=1024):
#         resize_transform = albumentations.Compose([
#             albumentations.Resize(height=image_size, width=image_size, interpolation=cv2.INTER_AREA, p=1),
#         ])
#         return resize_transform
# 
# # Directory setup
# PROJECT_ROOT_DIR = os.path.join(r"C:\Users\ambra\OneDrive - TU Eindhoven\Documents\uni\y3\DC3")
# logger.info(f"Root directory for project: {PROJECT_ROOT_DIR}")
# 
# # Correct data folder
# data_folder = os.path.join(PROJECT_ROOT_DIR, 'combined', 'benthic_datasets', 'point_labels', 'SEAVIEW')
# logger.info(f"Data folder for images and masks: {data_folder}")
# 
# # Initialize DataLoader
# data_loader = DataLoader(data_folder=data_folder, transform=DataLoader.get_resize_transform(image_size=1024))
# 
# # Setting a working folder
# working_folder = os.path.join(PROJECT_ROOT_DIR, 'temp', 'SAM-B1-RAND-300-F')
# logger.info(f"Working folder: {working_folder}")


In [4]:
import logging
import os
import pandas as pd
import cv2
import albumentations

# Set up logging
console_formatter = logging.Formatter('%(message)s')
console_handler = logging.StreamHandler()
console_handler.setFormatter(console_formatter)
logger = logging.getLogger('Label-Propagation')
logger.handlers.clear()
logger.propagate = False
logger.addHandler(console_handler)
logger.setLevel(logging.INFO)

class DataLoader:
    """
    DataLoader class for loading, processing, and transforming image, mask, and tabular data.
    """

    def __init__(self, image_folders, mask_folders, tabular_folder, transform=None):
        self.image_folders = image_folders
        self.mask_folders = mask_folders
        self.tabular_folder = tabular_folder
        self.transform = transform
        self.dataset = {}
        
        self.load_images()
        self.load_masks()
        self.load_tabular_data()

    def load_images(self):
        # Load image paths from the specified folders without loading the images yet
        self.image_paths = []
        for folder in self.image_folders:
            for img_file in os.listdir(folder):
                if img_file.endswith(('.png', '.jpg', '.jpeg')):  # Image extensions
                    self.image_paths.append(os.path.join(folder, img_file))
        logger.info(f"Loaded {len(self.image_paths)} images.")

    def load_masks(self):
        # Load mask paths from the specified folders
        self.mask_paths = []
        for folder in self.mask_folders:
            for mask_file in os.listdir(folder):
                if mask_file.endswith(('.png', '.jpg', '.jpeg')):  # Mask extensions
                    self.mask_paths.append(os.path.join(folder, mask_file))
        logger.info(f"Loaded {len(self.mask_paths)} masks.")

    def load_tabular_data(self):
        # Load tabular annotation data
        self.annotations = {}
        for annotation_file in os.listdir(self.tabular_folder):
            if annotation_file.startswith('annotations_') and annotation_file.endswith('.csv'):
                annotation_df = pd.read_csv(os.path.join(self.tabular_folder, annotation_file))
                # Store annotations by quadratid
                for quadratid in annotation_df['quadratid'].unique():
                    self.annotations[quadratid] = annotation_df[annotation_df['quadratid'] == quadratid]

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = cv2.imread(img_path)
        img_name = os.path.basename(img_path)
        
        # Find corresponding mask and points if they exist
        mask = None
        for mask_path in self.mask_paths:
            if os.path.basename(mask_path) == img_name:  # Match by filename
                mask = cv2.imread(mask_path)
                break

        # Get points and labels
        keypoints = None
        point_labels = None
        for quadratid, annotation_df in self.annotations.items():
            if quadratid in img_name:
                keypoints = annotation_df[['x', 'y']].values
                point_labels = annotation_df['func_group'].values
                break

        # Apply transformation if defined
        if self.transform:
            transformed = self.transform(image=img, mask=mask) if mask is not None else self.transform(image=img)
            img, mask = transformed['image'], transformed.get('mask', None)

        return img, mask, keypoints, point_labels

    @staticmethod
    def get_resize_transform(image_size=1024):
        resize_transform = albumentations.Compose([
            albumentations.Resize(height=image_size, width=image_size, interpolation=cv2.INTER_AREA, p=1),
        ])
        return resize_transform

# Directory setup
PROJECT_ROOT_DIR = r"C:\Users\ambra\OneDrive - TU Eindhoven\Documents\uni\y3\DC3"
logger.info(f"Root directory for project: {PROJECT_ROOT_DIR}")

# Specify the folders containing the data
image_folders = [
    os.path.join(PROJECT_ROOT_DIR, 'combined', 'benthic_datasets', 'point_labels', 'SEAVIEW', 'PAC_AUS'),
    os.path.join(PROJECT_ROOT_DIR, 'combined', 'benthic_datasets', 'point_labels', 'SEAVIEW', 'PAC_IDN_PHL'),
    os.path.join(PROJECT_ROOT_DIR, 'combined', 'benthic_datasets', 'point_labels', 'SEAVIEW', 'PAC_USA'),
    os.path.join(PROJECT_ROOT_DIR, 'combined', 'benthic_datasets', 'point_labels', 'SEAVIEW', 'ATL'),
]

mask_folders = [
    os.path.join(PROJECT_ROOT_DIR, 'combined', 'benthic_datasets', 'mask_labels', 'reef_support', 'SEAVIEW_ATL', 'masks'),
    os.path.join(PROJECT_ROOT_DIR, 'combined', 'benthic_datasets', 'mask_labels', 'reef_support', 'SEAVIEW_ATL', 'images'),
]

tabular_folder = os.path.join(PROJECT_ROOT_DIR, 'combined', 'benthic_datasets', 'point_labels', 'SEAVIEW', 'tabular-data')

# Initialize DataLoader
data_loader = DataLoader(
    image_folders=image_folders,
    mask_folders=mask_folders,
    tabular_folder=tabular_folder,
    transform=DataLoader.get_resize_transform(image_size=1024)
)

# Initialize DataLoader
data_loader = DataLoader(
    image_folders=image_folders,
    mask_folders=mask_folders,
    tabular_folder=tabular_folder,
    transform=DataLoader.get_resize_transform(image_size=1024)
)


# Initialize DataLoader
data_loader = DataLoader(
    image_folders=image_folders,
    mask_folders=mask_folders,
    tabular_folder=tabular_folder,
    transform=DataLoader.get_resize_transform(image_size=1024)
)

# Proceed with the rest of your analysis...


C:\Users\ambra\PycharmProjects\DC3_Group8\.venv\Lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.16). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
Root directory for project: C:\Users\ambra\OneDrive - TU Eindhoven\Documents\uni\y3\DC3
Loaded 6855 images.
Loaded 10268 masks.
Loaded 6855 images.
Loaded 10268 masks.
Loaded 6855 images.
Loaded 10268 masks.


Directory setup


In [5]:
# Root folder in Google Drive for this project
import os 
PROJECT_ROOT_DIR = os.path.join(r"C:\Users\ambra\OneDrive - TU Eindhoven\Documents\uni\y3\DC3")
logger.info(f"Root directory for project in Google Drive: {PROJECT_ROOT_DIR}")

Root directory for project in Google Drive: C:\Users\ambra\OneDrive - TU Eindhoven\Documents\uni\y3\DC3


In [6]:
import sys

# add the path where point label aware superpixels and custom packags is located
sys.path.append(os.path.join(PROJECT_ROOT_DIR, 'packages'))


Installations


In [7]:
import torch

print(torch.__version__)

# workaround to overcome long duration needed for installing torch_scatter
#!pip install pyg_lib torch_scatter -f https://data.pyg.org/whl/torch-{torch.2.4.1+cu124}.html
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.4.0+cu124.html

2.4.1+cu124
Looking in links: https://data.pyg.org/whl/torch-2.4.0+cu124.html



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
!pip install torchmetrics
!pip install timm



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Imports


In [9]:
import cv2
import yaml
import pandas as pd
import albumentations

from pathlib import Path

In [10]:
from labelmate.loader import LabelPropDataLoader
from labelmate.patchifier import SimplePatchifier
from labelmate.visualizer import visualize_output
from labelmate.evaluator import LabelPropEvaluator
from labelmate.propagator import SAMPointPromptsLabelProp

In [11]:
custom_modules = ['labelmate.loader', 'labelmate.visualizer',
                  'labelmate.evaluator', 'labelmate.propagator',
                  'labelmate.hypertuner', 'labelmate.patchifier',
                  ]
for module_name in custom_modules:
    logging.getLogger(module_name).setLevel(logging.INFO)
custom_modules = ['torchmetrics',
                  ]
for module_name in custom_modules:
    logging.getLogger(module_name).setLevel(logging.ERROR)
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

In [12]:
import torch
print("Is CUDA available?:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
print("CUDA current device:", torch.cuda.current_device())
print("CUDA device name:", torch.cuda.get_device_name(torch.cuda.current_device()))


Is CUDA available?: True
CUDA device count: 1
CUDA current device: 0
CUDA device name: NVIDIA GeForce RTX 3050 4GB Laptop GPU


Import SAM pre-trained weights

In [12]:
# dowload the pre-trained weights mentioned in GitHub and point to that file
SAM_WEIGHTS_PATH = r'C:/Users/ambra/OneDrive - TU Eindhoven/Documents/uni/y3/DC3/weights/sam'

SAM_WEIGHTS_FILE = 'sam_vit_h_4b8939.pth'
SAM_WEIGHTS_URL = f"https://dl.fbaipublicfiles.com/segment_anything/{SAM_WEIGHTS_FILE}"

#!wget {SAM_WEIGHTS_URL} -P {SAM_WEIGHTS_PATH}
import requests

# Ensure the directory exists
os.makedirs(SAM_WEIGHTS_PATH, exist_ok=True)

# Download the file
response = requests.get(SAM_WEIGHTS_URL)
if response.status_code == 200:
    file_path = os.path.join(SAM_WEIGHTS_PATH, SAM_WEIGHTS_FILE)
    with open(file_path, 'wb') as f:
        f.write(response.content)
    print(f"Weights downloaded and saved to {file_path}")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

Weights downloaded and saved to C:/Users/ambra/OneDrive - TU Eindhoven/Documents/uni/y3/DC3/weights/sam\sam_vit_h_4b8939.pth


Path variables

In [13]:
# Path to folder that contains images and dense masks
DATA_ROOT_DIR = os.path.join(PROJECT_ROOT_DIR,'combined', 'benthic_datasets', 'mask_labels', 'reef_support')
logger.info(f"Dataset (images and masks) will be accessed from: {DATA_ROOT_DIR}")

Dataset (images and masks) will be accessed from: C:\Users\ambra\OneDrive - TU Eindhoven\Documents\uni\y3\DC3\combined\benthic_datasets\mask_labels\reef_support


In [14]:
# Path to permanently store input images, input sparse masks in image format and output from label propagation
RESULTS_UPLOAD_PATH = os.path.join(PROJECT_ROOT_DIR, 'findings', 'SAM-B1-RAND-300-F')
logger.info(f"Results obtained will be saved to: {RESULTS_UPLOAD_PATH}")

Results obtained will be saved to: C:\Users\ambra\OneDrive - TU Eindhoven\Documents\uni\y3\DC3\findings\SAM-B1-RAND-300-F


In [15]:


# Function to load and preprocess images
#def preprocess_image(image_path, image_size=1024):
#    """Loads and resizes the image."""
#    image = cv2.imread(image_path)
#    transform = albumentations.Resize(height=image_size, width=image_size, interpolation=cv2.INTER_AREA)
#    transformed = transform(image=image)['image']
#    return transformed
#
#def create_labelmate_dataset(data_loader, point_labels_source='DENSE', point_labels_count=100):
#    """
#    Create dataset for label propagation by extracting information from the DataLoader.
#    
#    Args:
#    - data_loader: Instance of DataLoader containing images, masks, and tabular data.
#    - point_labels_source (str): The source of the point labels (default is 'DENSE').
#    - point_labels_count (int): Number of point labels to use (default is 100).
#    
#    Returns:
#    - dataset_df: DataFrame containing all relevant data for the label propagation process.
#    """
#    samples = []
#
#    for idx in range(len(data_loader)):
#        image_file_name = data_loader.img_names[idx]
#        img, mask, keypoints, point_labels = data_loader[idx]
#
#        sample = dict(
#            sample_id=image_file_name,
#            dataset_name=os.path.basename(data_loader.data_folder),  # assuming folder name is the dataset name
#            region=data_loader.data_folder.split('/')[-2],  # assuming folder structure follows region convention
#            image_path=image_file_name,
#            mask_path=image_file_name.replace('images', 'masks').replace('.jpg', '_mask.png'),  # adjust if needed
#            point_labels_source=point_labels_source,
#            point_labels_count=point_labels_count,
#        )
#        samples.append(sample)
#
#    dataset_df = pd.DataFrame(samples)
#    return dataset_df
#
#def get_resize_transform(image_size=1024):
#    resize_transform = \
#        albumentations.Compose(
#            [albumentations.Resize(
#                height=image_size,
#                width=image_size,
#                interpolation=cv2.INTER_AREA,
#                p=1,
#                ),
#            ]
#            )
#    return resize_transform

TypeError: DataLoader.__init__() got an unexpected keyword argument 'data_folder'

In [1]:
import os
import cv2
import albumentations
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torch
from sklearn.preprocessing import LabelEncoder
from segment_anything import SamPredictor, sam_model_registry
from pathlib import Path


class DataLoader:
    """
    DataLoader class for loading, processing, and transforming image, mask, and tabular data.
    """
    def __init__(self, img_folder, mask_folder, tabular_folder, transform=None):
        self.img_folder = img_folder
        self.mask_folder = mask_folder
        self.tabular_folder = tabular_folder
        self.transform = transform

        # Load image and mask data
        self.images = sorted([f for f in os.listdir(self.img_folder) if f.endswith(('.jpg', '.png'))])
        self.masks = sorted([f for f in os.listdir(self.mask_folder) if f.endswith(('.jpg', '.png'))])

        # Print image and mask names for debugging
        print(f"Image Names: {self.images}")
        print(f"Mask Names: {self.masks}")

        # Create sets for comparison, stripping _mask suffix from masks
        img_names_set = set([os.path.splitext(f)[0] for f in self.images])
        mask_names_set = set([os.path.splitext(f)[0].replace('_mask', '') for f in self.masks])  # Ensure we handle '_mask'

        # Find corresponding pairs
        valid_img_names = img_names_set & mask_names_set
        self.images = [f for f in self.images if os.path.splitext(f)[0] in valid_img_names]
        self.masks = [f"{os.path.splitext(f)[0]}_mask.png" for f in self.images]  # Create the corresponding mask filenames

        # Print filtered names for debugging
        print(f"Filtered Image Names: {self.images}")
        print(f"Filtered Mask Names: {self.masks}")

        # Create a dataset dictionary with image paths as keys and image, mask data
        self.dataset = {}
        for img_p in tqdm(self.images, desc="Loading images and masks"):
            img_name = os.path.splitext(img_p)[0]
            img = cv2.imread(os.path.join(self.img_folder, img_p))
            mask_path = os.path.join(self.mask_folder, img_name + '_mask.png')  # Use .png as per your example
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

            # Check if the mask is loaded correctly
            if mask is None:
                print(f"Warning: Mask for {img_name} could not be loaded. Mask path: {mask_path}")
                continue

            # Ensure images and masks have the same dimensions
            if img is not None and img.shape[:2] != mask.shape[:2]:
                print(f"Image and mask sizes do not match for {img_p}. Resizing mask.")
                mask = cv2.resize(mask, (img.shape[1], img.shape[0]))

            # Store the data
            self.dataset[img_name] = {'image': img, 'mask': mask}

        self.img_names = list(self.dataset.keys())

        # Initialize the LabelEncoder
        self.label_encoder = LabelEncoder()

        # Load tabular annotation data and add to the dataset
        for annotation_df_path in tqdm(os.listdir(self.tabular_folder), desc="Loading tabular data"):
            if not annotation_df_path.startswith('annotation'):
                continue

            annotation_df = pd.read_csv(os.path.join(self.tabular_folder, annotation_df_path))

            # Print column names for debugging
            print(f"Columns in {annotation_df_path}: {annotation_df.columns}")

            # Check if the expected 'quadratid' column exists, otherwise adjust
            if 'quadratid' not in annotation_df.columns:
                raise KeyError(f"'quadratid' column not found in {annotation_df_path}. Available columns: {annotation_df.columns}")

            annotation_df['quadratid'] = annotation_df['quadratid'].astype(str)

            # Fit the LabelEncoder on the func_group to convert them into integers
            self.label_encoder.fit(annotation_df['func_group'].astype(str))  # Make sure to convert to string

            # Match tabular data with image data in the dataset
            for quadratid in annotation_df['quadratid'].unique():
                if quadratid not in self.dataset:
                    continue

                # Ensure no duplicate quadratid with different data exists
                if 'points' in self.dataset[quadratid]:
                    raise Exception("Same quadrat, different datasets!")

                # Add points and point labels to the dataset
                self.dataset[quadratid]['points'] = annotation_df[annotation_df['quadratid'] == quadratid][['x', 'y']].values
                point_labels = annotation_df[annotation_df['quadratid'] == quadratid]['func_group'].values
                
                # Transform labels to integers
                self.dataset[quadratid]['point_labels'] = self.label_encoder.transform(point_labels)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img_dict = self.dataset[self.img_names[idx]]
        img, mask, keypoints, point_labels = img_dict['image'], img_dict['mask'], img_dict.get('points', None), img_dict.get('point_labels', None)

        if self.transform:
            transform = self.get_resize_transform(image_size=1024)
            transformed = transform(image=img, mask=mask)
            img, mask = transformed['image'], transformed['mask']

        return img, mask, keypoints, point_labels

    @staticmethod
    def get_resize_transform(image_size=1024):
        return albumentations.Compose([
            albumentations.Resize(
                height=image_size,
                width=image_size,
                interpolation=cv2.INTER_AREA,
                p=1,
            ),
        ])


# Helper functions for SAM
def load_sam_model(sam_weights_path):
    sam = sam_model_registry['vit_h'](checkpoint=sam_weights_path)
    predictor = SamPredictor(sam)
    return predictor


def main_pipeline(img_folder, mask_folder, tabular_folder, sam_weights_path):
    # Initialize the DataLoader with your dataset
    data_loader = DataLoader(
        img_folder=img_folder,
        mask_folder=mask_folder,
        tabular_folder=tabular_folder,
        transform=DataLoader.get_resize_transform(image_size=1024)
    )

    # Load the SAM model
    sam_predictor = load_sam_model(sam_weights_path)

    # Progress bar for the main pipeline
    progress_bar = tqdm(range(len(data_loader)), desc="Processing images")

    for idx in progress_bar:
        img, mask, keypoints, point_labels = data_loader[idx]

        # Check types of img and mask
        print(f"Processing image {idx}: {type(img)}, shape: {img.shape}, type(mask): {type(mask)}, shape: {mask.shape}")

        # Check if keypoints and point_labels are not None
        if keypoints is None or point_labels is None:
            print(f"Warning: Keypoints or point_labels are None for image {data_loader.img_names[idx]}. Skipping.")
            continue

        # Use SAM to predict masks
        sam_predictor.set_image(img)

        try:
            predicted_masks = sam_predictor.predict(point_coords=keypoints, point_labels=point_labels)
        except Exception as e:
            print(f"Error processing image {idx}: {str(e)}")
            continue  # Skip this iteration if there's an error

        # Update progress description
        progress_bar.set_description(f"Processing image {idx + 1}/{len(data_loader)}")

    print("Pipeline completed!")


# Example usage
img_folder = r'C:\Users\ambra\OneDrive - TU Eindhoven\Documents\uni\y3\DC3\combined\benthic_datasets\point_labels\SEAVIEW\ATL'
mask_folder = r'C:\Users\ambra\OneDrive - TU Eindhoven\Documents\uni\y3\DC3\combined\benthic_datasets\mask_labels\reef_support\SEAVIEW_ATL\masks_stitched'
tabular_folder = r'C:\Users\ambra\OneDrive - TU Eindhoven\Documents\uni\y3\DC3\combined\benthic_datasets\point_labels\SEAVIEW\tabular-data'

sam_weights_path = Path('C:/Users/ambra/PycharmProjects/DC3_Group8/weights/sam/sam_vit_h_4b8939.pth')

# Run the pipeline
main_pipeline(img_folder, mask_folder, tabular_folder, sam_weights_path)

# Offload GPU memory after processing
torch.cuda.empty_cache()



C:\Users\ambra\PycharmProjects\DC3_Group8\.venv\Lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.16). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Image Names: ['17001652802.jpg', '17001669902.jpg', '17001737702.jpg', '17001738002.jpg', '17001738102.jpg', '17001738202.jpg', '17001738302.jpg', '17001738402.jpg', '17001738502.jpg', '17001738602.jpg', '17001738702.jpg', '17001738802.jpg', '17001738902.jpg', '17001739002.jpg', '17001739102.jpg', '17001739202.jpg', '17001739302.jpg', '17001739402.jpg', '17001739502.jpg', '17001739602.jpg', '17001739702.jpg', '17001739802.jpg', '17001739902.jpg', '17001740002.jpg', '17001740102.jpg', '17001740202.jpg', '17001740302.jpg', '17001740402.jpg', '17001740502.jpg', '17001740602.jpg', '17001740702.jpg', '17001740802.jpg', '17001740902.jpg', '17001745802.jpg', '17002336001.jpg', '17002369001.jpg', '17002409801.jpg', '17004348102.jpg', '17004348202.jpg', '17004348302.jpg', '17004348402.jpg', '17004348502.jpg', '17004348602.jpg', '17004348702.jpg', '17004348802.jpg', '17004348902.jpg', '17004349002.jpg', '17004349102.jpg', '17004349202.jpg', '17004349302.jpg', '17004349402.jpg', '17004349502.jpg'

Loading images and masks:   0%|          | 0/659 [00:00<?, ?it/s]

Loading tabular data:   0%|          | 0/18 [00:00<?, ?it/s]

Columns in annotations_ATL.csv: Index(['quadratid', 'y', 'x', 'label_name', 'label', 'func_group', 'method',
       'data_set'],
      dtype='object')
Columns in annotations_IND_CHA.csv: Index(['quadratid', 'y', 'x', 'label_name', 'label', 'func_group', 'method',
       'data_set'],
      dtype='object')
Columns in annotations_IND_MDV.csv: Index(['quadratid', 'y', 'x', 'label_name', 'label', 'func_group', 'method',
       'data_set'],
      dtype='object')
Columns in annotations_PAC_AUS.csv: Index(['quadratid', 'y', 'x', 'label_name', 'label', 'func_group', 'method',
       'data_set'],
      dtype='object')
Columns in annotations_PAC_IDN_PHL.csv: Index(['quadratid', 'y', 'x', 'label_name', 'label', 'func_group', 'method',
       'data_set'],
      dtype='object')
Columns in annotations_PAC_SLB.csv: Index(['quadratid', 'y', 'x', 'label_name', 'label', 'func_group', 'method',
       'data_set'],
      dtype='object')
Columns in annotations_PAC_TLS.csv: Index(['quadratid', 'y', 'x', 'lab

C:\Users\ambra\PycharmProjects\DC3_Group8\.venv\Lib\site-packages\segment_anything\build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)

Processing images:   0%|          | 0/659 [00:00<?, ?it/s]

Processing image 0: <class 'numpy.ndarray'>, shape: (1024, 1024, 3), type(mask): <class 'numpy.ndarray'>, shape: (1024, 1024)
Processing image 1: <class 'numpy.ndarray'>, shape: (1024, 1024, 3), type(mask): <class 'numpy.ndarray'>, shape: (1024, 1024)
Processing image 2: <class 'numpy.ndarray'>, shape: (1024, 1024, 3), type(mask): <class 'numpy.ndarray'>, shape: (1024, 1024)
Processing image 3: <class 'numpy.ndarray'>, shape: (1024, 1024, 3), type(mask): <class 'numpy.ndarray'>, shape: (1024, 1024)
Processing image 4: <class 'numpy.ndarray'>, shape: (1024, 1024, 3), type(mask): <class 'numpy.ndarray'>, shape: (1024, 1024)
Processing image 5: <class 'numpy.ndarray'>, shape: (1024, 1024, 3), type(mask): <class 'numpy.ndarray'>, shape: (1024, 1024)
Processing image 6: <class 'numpy.ndarray'>, shape: (1024, 1024, 3), type(mask): <class 'numpy.ndarray'>, shape: (1024, 1024)
Processing image 7: <class 'numpy.ndarray'>, shape: (1024, 1024, 3), type(mask): <class 'numpy.ndarray'>, shape: (1024

Code in cell below looksonly at a fouth of the data rom the atlantic datasets(around 150 images) and implements evalution metric IoU

In [1]:
import os
import cv2
import albumentations
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import jaccard_score, accuracy_score, precision_score, recall_score, f1_score
from segment_anything import SamPredictor, sam_model_registry
from pathlib import Path


class DataLoader:
    """
    DataLoader class for loading, processing, and transforming image, mask, and tabular data.
    """
    def __init__(self, img_folder, mask_folder, tabular_folder, transform=None, load_fraction=0.25):
        self.img_folder = img_folder
        self.mask_folder = mask_folder
        self.tabular_folder = tabular_folder
        self.transform = transform
        self.load_fraction = load_fraction

        # Load image and mask data
        self.images = sorted([f for f in os.listdir(self.img_folder) if f.endswith(('.jpg', '.png'))])
        self.masks = sorted([f for f in os.listdir(self.mask_folder) if f.endswith(('.jpg', '.png'))])

        # Print image and mask names for debugging
        print(f"Image Names: {self.images}")
        print(f"Mask Names: {self.masks}")

        # Create sets for comparison, stripping _mask suffix from masks
        img_names_set = set([os.path.splitext(f)[0] for f in self.images])
        mask_names_set = set([os.path.splitext(f)[0].replace('_mask', '') for f in self.masks])  # Ensure we handle '_mask'

        # Find corresponding pairs
        valid_img_names = img_names_set & mask_names_set
        self.images = [f for f in self.images if os.path.splitext(f)[0] in valid_img_names]
        self.masks = [f"{os.path.splitext(f)[0]}_mask.png" for f in self.images]  # Create the corresponding mask filenames
        
        # Limit the number of images and masks loaded
        num_images_to_load = int(len(self.images) * self.load_fraction)
        self.images = self.images[:num_images_to_load]
        self.masks = self.masks[:num_images_to_load]

        # Print filtered names for debugging
        print(f"Filtered Image Names: {self.images}")
        print(f"Filtered Mask Names: {self.masks}")

        # Create a dataset dictionary with image paths as keys and image, mask data
        self.dataset = {}
        for img_p in tqdm(self.images, desc="Loading images and masks"):
            img_name = os.path.splitext(img_p)[0]
            img = cv2.imread(os.path.join(self.img_folder, img_p))
            mask_path = os.path.join(self.mask_folder, img_name + '_mask.png')  # Use .png as per your example
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

            # Check if the mask is loaded correctly
            if mask is None:
                print(f"Warning: Mask for {img_name} could not be loaded. Mask path: {mask_path}")
                continue

            # Ensure images and masks have the same dimensions
            if img is not None and img.shape[:2] != mask.shape[:2]:
                print(f"Image and mask sizes do not match for {img_p}. Resizing mask.")
                mask = cv2.resize(mask, (img.shape[1], img.shape[0]))

            # Store the data
            self.dataset[img_name] = {'image': img, 'mask': mask}

        self.img_names = list(self.dataset.keys())

        # Initialize the LabelEncoder
        self.label_encoder = LabelEncoder()

        # Load tabular annotation data and add to the dataset
        for annotation_df_path in tqdm(os.listdir(self.tabular_folder), desc="Loading tabular data"):
            if not annotation_df_path.startswith('annotation'):
                continue

            annotation_df = pd.read_csv(os.path.join(self.tabular_folder, annotation_df_path))

            # Print column names for debugging
            print(f"Columns in {annotation_df_path}: {annotation_df.columns}")

            # Check if the expected 'quadratid' column exists, otherwise adjust
            if 'quadratid' not in annotation_df.columns:
                raise KeyError(f"'quadratid' column not found in {annotation_df_path}. Available columns: {annotation_df.columns}")

            annotation_df['quadratid'] = annotation_df['quadratid'].astype(str)

            # Fit the LabelEncoder on the func_group to convert them into integers
            self.label_encoder.fit(annotation_df['func_group'].astype(str))  # Make sure to convert to string

            # Match tabular data with image data in the dataset
            for quadratid in annotation_df['quadratid'].unique():
                if quadratid not in self.dataset:
                    continue

                # Ensure no duplicate quadratid with different data exists
                if 'points' in self.dataset[quadratid]:
                    raise Exception("Same quadrat, different datasets!")

                # Add points and point labels to the dataset
                self.dataset[quadratid]['points'] = annotation_df[annotation_df['quadratid'] == quadratid][['x', 'y']].values
                point_labels = annotation_df[annotation_df['quadratid'] == quadratid]['func_group'].values
                
                # Transform labels to integers
                self.dataset[quadratid]['point_labels'] = self.label_encoder.transform(point_labels)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        
        img_dict = self.dataset[self.img_names[idx]]
        img, mask, keypoints, point_labels = img_dict['image'], img_dict['mask'], img_dict.get('points', None), img_dict.get('point_labels', None)

        if self.transform:
            transform = self.get_resize_transform(image_size=1024)
            transformed = transform(image=img, mask=mask)
            img, mask = transformed['image'], transformed['mask']

        return img, mask, keypoints, point_labels

    @staticmethod
    def get_resize_transform(image_size=1024):
        return albumentations.Compose([
            albumentations.Resize(
                height=image_size,
                width=image_size,
                interpolation=cv2.INTER_AREA,
                p=1,
            ),
        ])


# Helper functions for SAM
def load_sam_model(sam_weights_path):
    sam = sam_model_registry['vit_h'](checkpoint=sam_weights_path)
    predictor = SamPredictor(sam)
    return predictor


# Metric calculation functions
def calculate_iou(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return jaccard_score(y_true, y_pred, average='binary')

def calculate_pixel_accuracy(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return accuracy_score(y_true, y_pred)

def calculate_dice_coefficient(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    intersection = np.sum(y_true * y_pred)
    dice_score = (2 * intersection) / (np.sum(y_true) + np.sum(y_pred))
    return dice_score

def calculate_precision(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return precision_score(y_true, y_pred, average='binary')

def calculate_recall(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return recall_score(y_true, y_pred, average='binary')

def calculate_f1_score(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return f1_score(y_true, y_pred, average='binary')


from sklearn.metrics import jaccard_score, f1_score

def calculate_iou(true_mask, pred_mask, num_classes):
    """
    Calculate Intersection over Union (IoU) for multi-class segmentation.
    """
    iou_scores = []
    for class_id in range(num_classes):
        # Flatten the masks
        true_flat = (true_mask.flatten() == class_id).astype(int)
        pred_flat = (pred_mask.flatten() == class_id).astype(int)
        
        # Calculate IoU for each class
        iou = jaccard_score(true_flat, pred_flat, zero_division=0)
        iou_scores.append(iou)

    return np.mean(iou_scores)  # Return average IoU

def calculate_dice(true_mask, pred_mask, num_classes):
    """
    Calculate Dice Coefficient for multi-class segmentation.
    """
    dice_scores = []
    for class_id in range(num_classes):
        # Flatten the masks
        true_flat = (true_mask.flatten() == class_id).astype(int)
        pred_flat = (pred_mask.flatten() == class_id).astype(int)
        
        # Calculate Dice for each class
        dice = f1_score(true_flat, pred_flat, zero_division=0)
        dice_scores.append(dice)

    return np.mean(dice_scores)  # Return average Dice

def main_pipeline(img_folder, mask_folder, tabular_folder, sam_weights_path):
    # Initialize the DataLoader with your dataset
    data_loader = DataLoader(
        img_folder=img_folder,
        mask_folder=mask_folder,
        tabular_folder=tabular_folder,
        transform=DataLoader.get_resize_transform(image_size=1024)
    )

    # Load the SAM model
    sam_predictor = load_sam_model(sam_weights_path)

    # Progress bar for the main pipeline
    progress_bar = tqdm(range(len(data_loader)), desc="Processing images")

    # Lists to store metrics
    iou_scores = []
    dice_scores = []

    num_classes = 3  # Set this to the number of classes in your dataset

    for idx in progress_bar:
        img, mask, keypoints, point_labels = data_loader[idx]

        # Check types of img and mask
        print(f"Processing image {idx}: type(img)={type(img)}, shape(img)={img.shape}, type(mask)={type(mask)}, shape(mask)={mask.shape}")

        # Check if keypoints and point_labels are not None
        if keypoints is None or point_labels is None:
            print(f"Warning: Keypoints or point_labels are None for image {data_loader.img_names[idx]}. Skipping.")
            continue

        # Use SAM to predict masks
        sam_predictor.set_image(img)

        try:
            # Unpack predicted masks and any additional output (if any)
            predicted_masks, *additional_outputs = sam_predictor.predict(point_coords=keypoints, point_labels=point_labels)

            # Print shape of predicted masks for debugging
            if isinstance(predicted_masks, tuple):
                predicted_masks = predicted_masks[0]  # Extract the first element if it's a tuple

            print(f"Predicted masks shape: {predicted_masks.shape if predicted_masks is not None else 'None'}")

            # Check if the predicted masks are empty or None
            if predicted_masks is None or predicted_masks.size == 0:
                print(f"Warning: No masks predicted for image {data_loader.img_names[idx]}. Skipping.")
                continue  # Skip this iteration if there's no prediction

            # Convert predicted masks to uint8 if they are boolean
            if predicted_masks.dtype == np.bool_:
                predicted_masks = predicted_masks.astype(np.uint8) * 255  # Convert to uint8 format

            # Reduce to a single channel if necessary (e.g., take the maximum across channels)
            if predicted_masks.ndim == 3:  # If there are multiple channels
                predicted_masks = np.max(predicted_masks, axis=0)  # Or apply any other reduction logic (e.g., averaging)

            # Ensure predicted_masks is uint8 after max operation
            predicted_masks = (predicted_masks > 0).astype(np.uint8) * 255  # Thresholding to get binary mask

            # Resize predicted mask if necessary
            if predicted_masks.shape != mask.shape:
                print(f"Resizing predicted masks from {predicted_masks.shape} to {mask.shape}")
                predicted_masks = cv2.resize(predicted_masks, (mask.shape[1], mask.shape[0]))

            # Calculate metrics
            iou = calculate_iou(mask, predicted_masks, num_classes)

            # Store metrics
            iou_scores.append(iou)
 

        except Exception as e:
            print(f"Error processing image {idx}: {str(e)}")
            continue  # Skip this iteration if there's an error

        # Update progress description
        progress_bar.set_description(f"Processing image {idx + 1}/{len(data_loader)}")

    # Print average metrics
    avg_iou = np.mean(iou_scores) if iou_scores else 0

    print(f"Average IoU: {avg_iou:.4f}")
    print("Pipeline completed!")







# Example usage
img_folder = r'C:\Users\ambra\OneDrive - TU Eindhoven\Documents\uni\y3\DC3\combined\benthic_datasets\point_labels\SEAVIEW\ATL'
mask_folder = r'C:\Users\ambra\OneDrive - TU Eindhoven\Documents\uni\y3\DC3\combined\benthic_datasets\mask_labels\reef_support\SEAVIEW_ATL\masks_stitched'
tabular_folder = r'C:\Users\ambra\OneDrive - TU Eindhoven\Documents\uni\y3\DC3\combined\benthic_datasets\point_labels\SEAVIEW\tabular-data'

sam_weights_path = Path('C:/Users/ambra/PycharmProjects/DC3_Group8/weights/sam/sam_vit_h_4b8939.pth')

# Run the pipeline
main_pipeline(img_folder, mask_folder, tabular_folder, sam_weights_path)

# Offload GPU memory after processing
torch.cuda.empty_cache()


C:\Users\ambra\PycharmProjects\DC3_Group8\.venv\Lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.16). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Image Names: ['17001652802.jpg', '17001669902.jpg', '17001737702.jpg', '17001738002.jpg', '17001738102.jpg', '17001738202.jpg', '17001738302.jpg', '17001738402.jpg', '17001738502.jpg', '17001738602.jpg', '17001738702.jpg', '17001738802.jpg', '17001738902.jpg', '17001739002.jpg', '17001739102.jpg', '17001739202.jpg', '17001739302.jpg', '17001739402.jpg', '17001739502.jpg', '17001739602.jpg', '17001739702.jpg', '17001739802.jpg', '17001739902.jpg', '17001740002.jpg', '17001740102.jpg', '17001740202.jpg', '17001740302.jpg', '17001740402.jpg', '17001740502.jpg', '17001740602.jpg', '17001740702.jpg', '17001740802.jpg', '17001740902.jpg', '17001745802.jpg', '17002336001.jpg', '17002369001.jpg', '17002409801.jpg', '17004348102.jpg', '17004348202.jpg', '17004348302.jpg', '17004348402.jpg', '17004348502.jpg', '17004348602.jpg', '17004348702.jpg', '17004348802.jpg', '17004348902.jpg', '17004349002.jpg', '17004349102.jpg', '17004349202.jpg', '17004349302.jpg', '17004349402.jpg', '17004349502.jpg'

Loading images and masks:   0%|          | 0/164 [00:00<?, ?it/s]

Loading tabular data:   0%|          | 0/18 [00:00<?, ?it/s]

Columns in annotations_ATL.csv: Index(['quadratid', 'y', 'x', 'label_name', 'label', 'func_group', 'method',
       'data_set'],
      dtype='object')
Columns in annotations_IND_CHA.csv: Index(['quadratid', 'y', 'x', 'label_name', 'label', 'func_group', 'method',
       'data_set'],
      dtype='object')
Columns in annotations_IND_MDV.csv: Index(['quadratid', 'y', 'x', 'label_name', 'label', 'func_group', 'method',
       'data_set'],
      dtype='object')
Columns in annotations_PAC_AUS.csv: Index(['quadratid', 'y', 'x', 'label_name', 'label', 'func_group', 'method',
       'data_set'],
      dtype='object')
Columns in annotations_PAC_IDN_PHL.csv: Index(['quadratid', 'y', 'x', 'label_name', 'label', 'func_group', 'method',
       'data_set'],
      dtype='object')
Columns in annotations_PAC_SLB.csv: Index(['quadratid', 'y', 'x', 'label_name', 'label', 'func_group', 'method',
       'data_set'],
      dtype='object')
Columns in annotations_PAC_TLS.csv: Index(['quadratid', 'y', 'x', 'lab

C:\Users\ambra\PycharmProjects\DC3_Group8\.venv\Lib\site-packages\segment_anything\build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)

Processing images:   0%|          | 0/164 [00:00<?, ?it/s]

Processing image 0: type(img)=<class 'numpy.ndarray'>, shape(img)=(1024, 1024, 3), type(mask)=<class 'numpy.ndarray'>, shape(mask)=(1024, 1024)
Predicted masks shape: (3, 1024, 1024)
Processing image 1: type(img)=<class 'numpy.ndarray'>, shape(img)=(1024, 1024, 3), type(mask)=<class 'numpy.ndarray'>, shape(mask)=(1024, 1024)
Predicted masks shape: (3, 1024, 1024)
Processing image 2: type(img)=<class 'numpy.ndarray'>, shape(img)=(1024, 1024, 3), type(mask)=<class 'numpy.ndarray'>, shape(mask)=(1024, 1024)
Predicted masks shape: (3, 1024, 1024)
Processing image 3: type(img)=<class 'numpy.ndarray'>, shape(img)=(1024, 1024, 3), type(mask)=<class 'numpy.ndarray'>, shape(mask)=(1024, 1024)
Predicted masks shape: (3, 1024, 1024)
Processing image 4: type(img)=<class 'numpy.ndarray'>, shape(img)=(1024, 1024, 3), type(mask)=<class 'numpy.ndarray'>, shape(mask)=(1024, 1024)
Predicted masks shape: (3, 1024, 1024)
Processing image 5: type(img)=<class 'numpy.ndarray'>, shape(img)=(1024, 1024, 3), ty

Code below should separate data into 66? batches of size 10 each i haven't resolved all errors but this is like the base of the code with batches

In [1]:
import os
import cv2
import albumentations
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import jaccard_score, accuracy_score, precision_score, recall_score, f1_score
from segment_anything import SamPredictor, sam_model_registry
from pathlib import Path


class DataLoader:
    """
    DataLoader class for loading, processing, and transforming image, mask, and tabular data.
    """
    def __init__(self, img_folder, mask_folder, tabular_folder, transform=None, load_fraction=0.25):
        self.img_folder = img_folder
        self.mask_folder = mask_folder
        self.tabular_folder = tabular_folder
        self.transform = transform
        self.load_fraction = load_fraction

        # Load image and mask data
        self.images = sorted([f for f in os.listdir(self.img_folder) if f.endswith(('.jpg', '.png'))])
        self.masks = sorted([f for f in os.listdir(self.mask_folder) if f.endswith(('.jpg', '.png'))])

        # Print image and mask names for debugging
        print(f"Image Names: {self.images}")
        print(f"Mask Names: {self.masks}")

        # Create sets for comparison, stripping _mask suffix from masks
        img_names_set = set([os.path.splitext(f)[0] for f in self.images])
        mask_names_set = set([os.path.splitext(f)[0].replace('_mask', '') for f in self.masks])  # Ensure we handle '_mask'

        # Find corresponding pairs
        valid_img_names = img_names_set & mask_names_set
        self.images = [f for f in self.images if os.path.splitext(f)[0] in valid_img_names]
        self.masks = [f"{os.path.splitext(f)[0]}_mask.png" for f in self.images]  # Create the corresponding mask filenames
        
        # Limit the number of images and masks loaded
        num_images_to_load = int(len(self.images) * self.load_fraction)
        self.images = self.images[:num_images_to_load]
        self.masks = self.masks[:num_images_to_load]

        # Print filtered names for debugging
        print(f"Filtered Image Names: {self.images}")
        print(f"Filtered Mask Names: {self.masks}")

        # Create a dataset dictionary with image paths as keys and image, mask data
        self.dataset = {}
        for img_p in tqdm(self.images, desc="Loading images and masks"):
            img_name = os.path.splitext(img_p)[0]
            img = cv2.imread(os.path.join(self.img_folder, img_p))
            mask_path = os.path.join(self.mask_folder, img_name + '_mask.png')  # Use .png as per your example
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

            # Check if the mask is loaded correctly
            if mask is None:
                print(f"Warning: Mask for {img_name} could not be loaded. Mask path: {mask_path}")
                continue

            # Ensure images and masks have the same dimensions
            if img is not None and img.shape[:2] != mask.shape[:2]:
                print(f"Image and mask sizes do not match for {img_p}. Resizing mask.")
                mask = cv2.resize(mask, (img.shape[1], img.shape[0]))

            # Store the data
            self.dataset[img_name] = {'image': img, 'mask': mask}

        self.img_names = list(self.dataset.keys())

        # Initialize the LabelEncoder
        self.label_encoder = LabelEncoder()

        # Load tabular annotation data and add to the dataset
        for annotation_df_path in tqdm(os.listdir(self.tabular_folder), desc="Loading tabular data"):
            if not annotation_df_path.startswith('annotation'):
                continue

            annotation_df = pd.read_csv(os.path.join(self.tabular_folder, annotation_df_path))

            # Print column names for debugging
            print(f"Columns in {annotation_df_path}: {annotation_df.columns}")

            # Check if the expected 'quadratid' column exists, otherwise adjust
            if 'quadratid' not in annotation_df.columns:
                raise KeyError(f"'quadratid' column not found in {annotation_df_path}. Available columns: {annotation_df.columns}")

            annotation_df['quadratid'] = annotation_df['quadratid'].astype(str)

            # Fit the LabelEncoder on the func_group to convert them into integers
            self.label_encoder.fit(annotation_df['func_group'].astype(str))  # Make sure to convert to string

            # Match tabular data with image data in the dataset
            for quadratid in annotation_df['quadratid'].unique():
                if quadratid not in self.dataset:
                    continue

                # Ensure no duplicate quadratid with different data exists
                if 'points' in self.dataset[quadratid]:
                    raise Exception("Same quadrat, different datasets!")

                # Add points and point labels to the dataset
                self.dataset[quadratid]['points'] = annotation_df[annotation_df['quadratid'] == quadratid][['x', 'y']].values
                point_labels = annotation_df[annotation_df['quadratid'] == quadratid]['func_group'].values
                
                # Transform labels to integers
                self.dataset[quadratid]['point_labels'] = self.label_encoder.transform(point_labels)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        
        img_dict = self.dataset[self.img_names[idx]]
        img, mask, keypoints, point_labels = img_dict['image'], img_dict['mask'], img_dict.get('points', None), img_dict.get('point_labels', None)

        if self.transform:
            transform = self.get_resize_transform(image_size=1024)
            transformed = transform(image=img, mask=mask)
            img, mask = transformed['image'], transformed['mask']

        return img, mask, keypoints, point_labels

    @staticmethod
    def get_resize_transform(image_size=1024):
        return albumentations.Compose([
            albumentations.Resize(
                height=image_size,
                width=image_size,
                interpolation=cv2.INTER_AREA,
                p=1,
            ),
        ])


# Helper functions for SAM
def load_sam_model(sam_weights_path):
    sam = sam_model_registry['vit_h'](checkpoint=sam_weights_path)
    predictor = SamPredictor(sam)
    return predictor


# Metric calculation functions
def calculate_iou(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return jaccard_score(y_true, y_pred, average='binary')

def calculate_pixel_accuracy(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return accuracy_score(y_true, y_pred)

def calculate_dice_coefficient(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    intersection = np.sum(y_true * y_pred)
    dice_score = (2 * intersection) / (np.sum(y_true) + np.sum(y_pred))
    return dice_score

def calculate_precision(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return precision_score(y_true, y_pred, average='binary')

def calculate_recall(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return recall_score(y_true, y_pred, average='binary')

def calculate_f1_score(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return f1_score(y_true, y_pred, average='binary')


def main_pipeline(img_folder, mask_folder, tabular_folder, sam_weights_path):
    # Initialize the DataLoader with your dataset
    data_loader = DataLoader(
        img_folder=img_folder,
        mask_folder=mask_folder,
        tabular_folder=tabular_folder,
        transform=DataLoader.get_resize_transform(image_size=1024)
    )

    # Load the SAM model
    sam_predictor = load_sam_model(sam_weights_path)

    # Lists to hold the evaluation metrics
    iou_scores = []
    accuracy_scores = []
    dice_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []

    # Progress bar for the main pipeline
    progress_bar = tqdm(range(len(data_loader)), desc="Processing images")
    
    for idx in progress_bar:
        img, mask, keypoints, point_labels = data_loader[idx]

        # Check types of img and mask
        print(f"Processing image {idx}: {type(img)}, shape: {img.shape}, type(mask): {type(mask)}, shape: {mask.shape}")

        # Check if keypoints and point_labels are not None
        if keypoints is None or point_labels is None:
            print(f"Warning: Keypoints or point_labels are None for image {data_loader.img_names[idx]}. Skipping.")
            continue

        # Use SAM to predict masks
        sam_predictor.set_image(img)

        try:
            predicted_masks = sam_predictor.predict(point_coords=keypoints, point_labels=point_labels)
        except Exception as e:
            print(f"Error processing image {idx}: {str(e)}")
            continue  # Skip this iteration if there's an error

        predicted_mask = predicted_masks[0].astype(np.uint8)

        # Resize predicted mask if necessary
        if predicted_mask.shape != mask.shape:
            predicted_mask = cv2.resize(predicted_mask, (mask.shape[1], mask.shape[0]))

        # Calculate metrics
        # Calculate metrics
        iou = calculate_iou(mask, predicted_mask)
        accuracy = calculate_pixel_accuracy(mask, predicted_mask)
        dice = calculate_dice_coefficient(mask, predicted_mask)
        precision = calculate_precision(mask, predicted_mask)
        recall = calculate_recall(mask, predicted_mask)
        f1 = calculate_f1_score(mask, predicted_mask)

        # Store the scores
        iou_scores.append(iou)
        accuracy_scores.append(accuracy)
        dice_scores.append(dice)
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)

        # Update progress description
        progress_bar.set_description(f"Processing image {idx + 1}/{len(data_loader)}")

    # Calculate and print the average of each metric
    print("Evaluation Metrics:")
    print(f"Mean IoU: {np.mean(iou_scores):.4f}")
    print(f"Mean Accuracy: {np.mean(accuracy_scores):.4f}")
    print(f"Mean Dice Coefficient: {np.mean(dice_scores):.4f}")
    print(f"Mean Precision: {np.mean(precision_scores):.4f}")
    print(f"Mean Recall: {np.mean(recall_scores):.4f}")
    print(f"Mean F1 Score: {np.mean(f1_scores):.4f}")

    print("Pipeline completed!")


# Example usage
img_folder = r'C:\Users\ambra\OneDrive - TU Eindhoven\Documents\uni\y3\DC3\combined\benthic_datasets\point_labels\SEAVIEW\ATL'
mask_folder = r'C:\Users\ambra\OneDrive - TU Eindhoven\Documents\uni\y3\DC3\combined\benthic_datasets\mask_labels\reef_support\SEAVIEW_ATL\masks_stitched'
tabular_folder = r'C:\Users\ambra\OneDrive - TU Eindhoven\Documents\uni\y3\DC3\combined\benthic_datasets\point_labels\SEAVIEW\tabular-data'

sam_weights_path = Path('C:/Users/ambra/PycharmProjects/DC3_Group8/weights/sam/sam_vit_h_4b8939.pth')

# Run the pipeline
main_pipeline(img_folder, mask_folder, tabular_folder, sam_weights_path)

# Offload GPU memory after processing
torch.cuda.empty_cache()


C:\Users\ambra\PycharmProjects\DC3_Group8\.venv\Lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.16). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Image Names: ['17001652802.jpg', '17001669902.jpg', '17001737702.jpg', '17001738002.jpg', '17001738102.jpg', '17001738202.jpg', '17001738302.jpg', '17001738402.jpg', '17001738502.jpg', '17001738602.jpg', '17001738702.jpg', '17001738802.jpg', '17001738902.jpg', '17001739002.jpg', '17001739102.jpg', '17001739202.jpg', '17001739302.jpg', '17001739402.jpg', '17001739502.jpg', '17001739602.jpg', '17001739702.jpg', '17001739802.jpg', '17001739902.jpg', '17001740002.jpg', '17001740102.jpg', '17001740202.jpg', '17001740302.jpg', '17001740402.jpg', '17001740502.jpg', '17001740602.jpg', '17001740702.jpg', '17001740802.jpg', '17001740902.jpg', '17001745802.jpg', '17002336001.jpg', '17002369001.jpg', '17002409801.jpg', '17004348102.jpg', '17004348202.jpg', '17004348302.jpg', '17004348402.jpg', '17004348502.jpg', '17004348602.jpg', '17004348702.jpg', '17004348802.jpg', '17004348902.jpg', '17004349002.jpg', '17004349102.jpg', '17004349202.jpg', '17004349302.jpg', '17004349402.jpg', '17004349502.jpg'

Loading images and masks:   0%|          | 0/164 [00:00<?, ?it/s]

Loading tabular data:   0%|          | 0/18 [00:00<?, ?it/s]

Columns in annotations_ATL.csv: Index(['quadratid', 'y', 'x', 'label_name', 'label', 'func_group', 'method',
       'data_set'],
      dtype='object')
Columns in annotations_IND_CHA.csv: Index(['quadratid', 'y', 'x', 'label_name', 'label', 'func_group', 'method',
       'data_set'],
      dtype='object')
Columns in annotations_IND_MDV.csv: Index(['quadratid', 'y', 'x', 'label_name', 'label', 'func_group', 'method',
       'data_set'],
      dtype='object')
Columns in annotations_PAC_AUS.csv: Index(['quadratid', 'y', 'x', 'label_name', 'label', 'func_group', 'method',
       'data_set'],
      dtype='object')
Columns in annotations_PAC_IDN_PHL.csv: Index(['quadratid', 'y', 'x', 'label_name', 'label', 'func_group', 'method',
       'data_set'],
      dtype='object')
Columns in annotations_PAC_SLB.csv: Index(['quadratid', 'y', 'x', 'label_name', 'label', 'func_group', 'method',
       'data_set'],
      dtype='object')
Columns in annotations_PAC_TLS.csv: Index(['quadratid', 'y', 'x', 'lab

C:\Users\ambra\PycharmProjects\DC3_Group8\.venv\Lib\site-packages\segment_anything\build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)

Processing images:   0%|          | 0/164 [00:00<?, ?it/s]

Processing image 0: <class 'numpy.ndarray'>, shape: (1024, 1024, 3), type(mask): <class 'numpy.ndarray'>, shape: (1024, 1024)


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:3789: error: (-215:Assertion failed) !dsize.empty() in function 'cv::hal::resize'


The code below has an attempt at batches and CUDA and it does run but it takes forever and based on debugging statements it might process data wrongly

In [ ]:
import os
import cv2
import albumentations
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import jaccard_score, accuracy_score, precision_score, recall_score, f1_score
from segment_anything import SamPredictor, sam_model_registry
from pathlib import Path


class DataLoader:
    """
    DataLoader class for loading, processing, and transforming image, mask, and tabular data.
    """
    def __init__(self, img_folder, mask_folder, tabular_folder, transform=None, load_fraction=0.25):
        self.img_folder = img_folder
        self.mask_folder = mask_folder
        self.tabular_folder = tabular_folder
        self.transform = transform
        self.load_fraction = load_fraction

        # Load image and mask data
        self.images = sorted([f for f in os.listdir(self.img_folder) if f.endswith(('.jpg', '.png'))])
        self.masks = sorted([f for f in os.listdir(self.mask_folder) if f.endswith(('.jpg', '.png'))])

        # Print image and mask names for debugging
        print(f"Image Names: {self.images}")
        print(f"Mask Names: {self.masks}")

        # Create sets for comparison, stripping _mask suffix from masks
        img_names_set = set([os.path.splitext(f)[0] for f in self.images])
        mask_names_set = set([os.path.splitext(f)[0].replace('_mask', '') for f in self.masks])  # Ensure we handle '_mask'

        # Find corresponding pairs
        valid_img_names = img_names_set & mask_names_set
        self.images = [f for f in self.images if os.path.splitext(f)[0] in valid_img_names]
        self.masks = [f"{os.path.splitext(f)[0]}_mask.png" for f in self.images]  # Create the corresponding mask filenames
        
        # Limit the number of images and masks loaded
        # num_images_to_load = int(len(self.images) * self.load_fraction)
        # self.images = self.images[:num_images_to_load]
        # self.masks = self.masks[:num_images_to_load]

        # Print filtered names for debugging
        print(f"Filtered Image Names: {self.images}")
        print(f"Filtered Mask Names: {self.masks}")

        # Create a dataset dictionary with image paths as keys and image, mask data
        self.dataset = {}
        for img_p in tqdm(self.images, desc="Loading images and masks"):
            img_name = os.path.splitext(img_p)[0]
            img = cv2.imread(os.path.join(self.img_folder, img_p))
            mask_path = os.path.join(self.mask_folder, img_name + '_mask.png')  # Use .png as per your example
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

            # Check if the mask is loaded correctly
            if mask is None:
                print(f"Warning: Mask for {img_name} could not be loaded. Mask path: {mask_path}")
                continue

            # Ensure images and masks have the same dimensions
            if img is not None and img.shape[:2] != mask.shape[:2]:
                print(f"Image and mask sizes do not match for {img_p}. Resizing mask.")
                mask = cv2.resize(mask, (img.shape[1], img.shape[0]))

            # Store the data
            self.dataset[img_name] = {'image': img, 'mask': mask}

        self.img_names = list(self.dataset.keys())

        # Initialize the LabelEncoder
        self.label_encoder = LabelEncoder()

        # Load tabular annotation data and add to the dataset
        for annotation_df_path in tqdm(os.listdir(self.tabular_folder), desc="Loading tabular data"):
            if not annotation_df_path.startswith('annotation'):
                continue

            annotation_df = pd.read_csv(os.path.join(self.tabular_folder, annotation_df_path))

            # Print column names for debugging
            print(f"Columns in {annotation_df_path}: {annotation_df.columns}")

            # Check if the expected 'quadratid' column exists, otherwise adjust
            if 'quadratid' not in annotation_df.columns:
                raise KeyError(f"'quadratid' column not found in {annotation_df_path}. Available columns: {annotation_df.columns}")

            annotation_df['quadratid'] = annotation_df['quadratid'].astype(str)

            # Fit the LabelEncoder on the func_group to convert them into integers
            self.label_encoder.fit(annotation_df['func_group'].astype(str))  # Make sure to convert to string

            # Match tabular data with image data in the dataset
            for quadratid in annotation_df['quadratid'].unique():
                if quadratid not in self.dataset:
                    continue

                # Ensure no duplicate quadratid with different data exists
                if 'points' in self.dataset[quadratid]:
                    raise Exception("Same quadrat, different datasets!")

                # Add points and point labels to the dataset
                self.dataset[quadratid]['points'] = annotation_df[annotation_df['quadratid'] == quadratid][['x', 'y']].values
                point_labels = annotation_df[annotation_df['quadratid'] == quadratid]['func_group'].values
                
                # Transform labels to integers
                self.dataset[quadratid]['point_labels'] = self.label_encoder.transform(point_labels)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        
        img_dict = self.dataset[self.img_names[idx]]
        img, mask, keypoints, point_labels = img_dict['image'], img_dict['mask'], img_dict.get('points', None), img_dict.get('point_labels', None)

        if self.transform:
            transform = self.get_resize_transform(image_size=1024)
            transformed = transform(image=img, mask=mask)
            img, mask = transformed['image'], transformed['mask']

        return img, mask, keypoints, point_labels

    @staticmethod
    def get_resize_transform(image_size=1024):
        return albumentations.Compose([
            albumentations.Resize(
                height=image_size,
                width=image_size,
                interpolation=cv2.INTER_AREA,
                p=1,
            ),
        ])


# Helper functions for SAM
def load_sam_model(sam_weights_path):
    sam = sam_model_registry['vit_h'](checkpoint=sam_weights_path)
    predictor = SamPredictor(sam)
    return predictor


# Metric calculation functions
def calculate_iou(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return jaccard_score(y_true, y_pred, average='binary')

def calculate_pixel_accuracy(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return accuracy_score(y_true, y_pred)

def calculate_dice_coefficient(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    intersection = np.sum(y_true * y_pred)
    dice_score = (2 * intersection) / (np.sum(y_true) + np.sum(y_pred))
    return dice_score

def calculate_precision(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return precision_score(y_true, y_pred, average='binary')

def calculate_recall(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return recall_score(y_true, y_pred, average='binary')

def calculate_f1_score(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return f1_score(y_true, y_pred, average='binary')


from sklearn.metrics import jaccard_score, f1_score

def calculate_iou(true_mask, pred_mask, num_classes):
    """
    Calculate Intersection over Union (IoU) for multi-class segmentation.
    """
    iou_scores = []
    for class_id in range(num_classes):
        # Flatten the masks
        true_flat = (true_mask.flatten() == class_id).astype(int)
        pred_flat = (pred_mask.flatten() == class_id).astype(int)
        
        # Calculate IoU for each class
        iou = jaccard_score(true_flat, pred_flat, zero_division=0)
        iou_scores.append(iou)

    return np.mean(iou_scores)  # Return average IoU

def calculate_dice(true_mask, pred_mask, num_classes):
    """
    Calculate Dice Coefficient for multi-class segmentation.
    """
    dice_scores = []
    for class_id in range(num_classes):
        # Flatten the masks
        true_flat = (true_mask.flatten() == class_id).astype(int)
        pred_flat = (pred_mask.flatten() == class_id).astype(int)
        
        # Calculate Dice for each class
        dice = f1_score(true_flat, pred_flat, zero_division=0)
        dice_scores.append(dice)

    return np.mean(dice_scores)  # Return average Dice

def batch_data_loader(data_loader, batch_size=10):
    """
    Generator function to load images, masks, and other data in batches.
    """
    batch = []
    for idx in range(len(data_loader)):
        batch.append(data_loader[idx])
        if len(batch) == batch_size or idx == len(data_loader) - 1:
            yield batch
            batch = []


import torch
import numpy as np
import cv2
from tqdm import tqdm

def pad_sequence(sequences, max_length=None):
    # Determine the maximum length if not provided
    if max_length is None:
        max_length = max(len(seq) for seq in sequences if seq is not None)

    # Pad sequences to max_length
    padded_sequences = []
    for seq in sequences:
        if seq is not None:  # Check if seq is not None
            # Handle 1D and 2D sequences
            if seq.dim() == 1:
                padding = torch.zeros((max_length - len(seq),), device=seq.device)  # 1D padding
                padded_sequences.append(torch.cat([seq, padding], dim=0))
            elif seq.dim() == 2:
                padding = torch.zeros((max_length - len(seq), seq.size(1)), device=seq.device)  # 2D padding
                padded_sequences.append(torch.cat([seq, padding], dim=0))
            else:
                print(f"Warning: Unsupported tensor dimension {seq.dim()} for padding.")
        else:
            print("Warning: One of the sequences is None.")
            padded_sequences.append(torch.zeros((max_length, 2), device=device))  # Adjust shape as needed

    return torch.stack(padded_sequences)




def main_pipeline(img_folder, mask_folder, tabular_folder, sam_weights_path, batch_size=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Initialize the DataLoader with your dataset
    data_loader = DataLoader(
        img_folder=img_folder,
        mask_folder=mask_folder,
        tabular_folder=tabular_folder,
        transform=DataLoader.get_resize_transform(image_size=1024)
    )

    # Load the SAM model
    sam_predictor = load_sam_model(sam_weights_path)

    # Progress bar for the main pipeline
    total_batches = (len(data_loader) + batch_size - 1) // batch_size
    progress_bar = tqdm(range(total_batches), desc="Processing batches")
    iou_scores = []
    dice_scores = []
    num_classes = 3  # Set this to the number of classes in your dataset
    
    for batch_idx, batch in enumerate(batch_data_loader(data_loader, batch_size=batch_size)):
        batch_images = []
        batch_masks = []
        batch_keypoints = []
        batch_point_labels = []

        # Unpack batch data and move to GPU
        for img, mask, keypoints, point_labels in batch:
            img = torch.tensor(img).to(device)  # Move to GPU
            mask = torch.tensor(mask).to(device)  # Move to GPU

            # Ensure img is a NumPy array and has the correct number of channels
            if isinstance(img, torch.Tensor):
                img = img.cpu().numpy()

            # Check and adjust image shape
            print(f"Processing image {batch_idx}: type(img)={type(img)}, shape(img)={img.shape}, type(mask)={type(mask)}, shape(mask)={mask.shape}")
            if img.shape[-1] > 4:  # If more than 4 channels
                print(f"Reducing image from {img.shape[-1]} to 4 channels.")
                img = img[:, :, :4]  # Adjust this based on your requirements

            # Normalize image to [0, 1]
            img = img.astype(np.float32) / 255.0

            # Collect data for processing
            batch_images.append(img)
            batch_masks.append(mask)
            batch_keypoints.append(keypoints)
            batch_point_labels.append(point_labels)

        # Lists to store metrics
        # Process each image in the batch
        for idx in range(len(batch_images)):
            img = batch_images[idx]
            mask = batch_masks[idx]

            keypoints = batch_keypoints[idx]
            point_labels = batch_point_labels[idx]

            # Check if keypoints and point_labels are not None
            if keypoints is None or point_labels is None:
                print(f"Warning: Keypoints or point_labels are None for image {data_loader.img_names[idx]}. Skipping.")
                continue

            # Use SAM to predict masks
            sam_predictor.set_image(img)

            try:
                # Unpack predicted masks and any additional output (if any)
                predicted_masks, *additional_outputs = sam_predictor.predict(point_coords=keypoints, point_labels=point_labels)

                # Print shape of predicted masks for debugging
                if isinstance(predicted_masks, tuple):
                    predicted_masks = predicted_masks[0]  # Extract the first element if it's a tuple

                print(f"Predicted masks shape: {predicted_masks.shape if predicted_masks is not None else 'None'}")

                # Check if the predicted masks are empty or None
                if predicted_masks is None or predicted_masks.size == 0:
                    print(f"Warning: No masks predicted for image {data_loader.img_names[idx]}. Skipping.")
                    continue  # Skip this iteration if there's no prediction

                # Convert predicted masks to uint8 if they are boolean
                if isinstance(predicted_masks, torch.Tensor):
                    predicted_masks = predicted_masks.cpu().numpy()  # Convert to NumPy array if needed
                if predicted_masks.dtype == np.bool_:
                    predicted_masks = (predicted_masks.astype(np.uint8) * 255)  # Convert to uint8 format

                # Reduce to a single channel if necessary (e.g., take the maximum across channels)
                if predicted_masks.ndim == 3:  # If there are multiple channels
                    predicted_masks = np.max(predicted_masks, axis=0)  # Or apply any other reduction logic (e.g., averaging)

                # Ensure predicted_masks is uint8 after max operation
                predicted_masks = (predicted_masks > 0).astype(np.uint8) * 255  # Thresholding to get binary mask

                # Resize predicted mask if necessary
                if predicted_masks.shape != mask.shape:
                    print(f"Resizing predicted masks from {predicted_masks.shape} to {mask.shape}")
                    predicted_masks = cv2.resize(predicted_masks, (mask.shape[1], mask.shape[0]))

                # Calculate metrics
                iou = calculate_iou(mask.cpu().numpy(), predicted_masks, num_classes)  # Convert mask to NumPy
                dice = calculate_dice(mask.cpu().numpy(), predicted_masks, num_classes)

                # Store metrics
                iou_scores.append(iou)
                dice_scores.append(dice)

            except Exception as e:
                print(f"Error processing image {idx}: {str(e)}")
                continue  # Skip this iteration if there's an error

        # Update progress description
        progress_bar.set_description(f"Processing batch {batch_idx + 1}/{total_batches}")

    # Print average metrics
    avg_iou = np.mean(iou_scores) if iou_scores else 0
    avg_dice = np.mean(dice_scores) if dice_scores else 0
    print(f"Average IoU: {avg_iou:.4f}, Average Dice: {avg_dice:.4f}")
    print("Pipeline completed!")






# Example usage
img_folder = r'C:\Users\ambra\OneDrive - TU Eindhoven\Documents\uni\y3\DC3\combined\benthic_datasets\point_labels\SEAVIEW\ATL'
mask_folder = r'C:\Users\ambra\OneDrive - TU Eindhoven\Documents\uni\y3\DC3\combined\benthic_datasets\mask_labels\reef_support\SEAVIEW_ATL\masks_stitched'
tabular_folder = r'C:\Users\ambra\OneDrive - TU Eindhoven\Documents\uni\y3\DC3\combined\benthic_datasets\point_labels\SEAVIEW\tabular-data'

sam_weights_path = Path('C:/Users/ambra/PycharmProjects/DC3_Group8/weights/sam/sam_vit_h_4b8939.pth')

# Run the pipeline
main_pipeline(img_folder, mask_folder, tabular_folder, sam_weights_path)

# Offload GPU memory after processing
torch.cuda.empty_cache()


Image Names: ['17001652802.jpg', '17001669902.jpg', '17001737702.jpg', '17001738002.jpg', '17001738102.jpg', '17001738202.jpg', '17001738302.jpg', '17001738402.jpg', '17001738502.jpg', '17001738602.jpg', '17001738702.jpg', '17001738802.jpg', '17001738902.jpg', '17001739002.jpg', '17001739102.jpg', '17001739202.jpg', '17001739302.jpg', '17001739402.jpg', '17001739502.jpg', '17001739602.jpg', '17001739702.jpg', '17001739802.jpg', '17001739902.jpg', '17001740002.jpg', '17001740102.jpg', '17001740202.jpg', '17001740302.jpg', '17001740402.jpg', '17001740502.jpg', '17001740602.jpg', '17001740702.jpg', '17001740802.jpg', '17001740902.jpg', '17001745802.jpg', '17002336001.jpg', '17002369001.jpg', '17002409801.jpg', '17004348102.jpg', '17004348202.jpg', '17004348302.jpg', '17004348402.jpg', '17004348502.jpg', '17004348602.jpg', '17004348702.jpg', '17004348802.jpg', '17004348902.jpg', '17004349002.jpg', '17004349102.jpg', '17004349202.jpg', '17004349302.jpg', '17004349402.jpg', '17004349502.jpg'




Loading images and masks:   0%|          | 0/659 [00:00<?, ?it/s]


Loading images and masks:   0%|          | 1/659 [00:00<01:20,  8.20it/s]


Loading images and masks:   0%|          | 2/659 [00:00<01:14,  8.86it/s]


Loading images and masks:   0%|          | 3/659 [00:00<01:10,  9.32it/s]


Loading images and masks:   1%|          | 5/659 [00:00<01:02, 10.44it/s]


Loading images and masks:   1%|          | 7/659 [00:00<00:59, 10.96it/s]


Loading images and masks:   1%|▏         | 9/659 [00:00<00:57, 11.27it/s]


Loading images and masks:   2%|▏         | 11/659 [00:01<01:10,  9.17it/s]


Loading images and masks:   2%|▏         | 12/659 [00:01<01:10,  9.22it/s]


Loading images and masks:   2%|▏         | 14/659 [00:01<01:01, 10.51it/s]


Loading images and masks:   2%|▏         | 16/659 [00:01<00:56, 11.44it/s]


Loading images and masks:   3%|▎         | 18/659 [00:01<00:53, 11.91it/s]


Loading images and masks:   3%|▎         | 20/659 [00:01<01:01, 10.45it/s]


Loading ima

Columns in annotations_ATL.csv: Index(['quadratid', 'y', 'x', 'label_name', 'label', 'func_group', 'method',
       'data_set'],
      dtype='object')





Loading tabular data:   6%|▌         | 1/18 [00:19<05:33, 19.61s/it]


Loading tabular data:  11%|█         | 2/18 [00:19<02:11,  8.20s/it]

Columns in annotations_IND_CHA.csv: Index(['quadratid', 'y', 'x', 'label_name', 'label', 'func_group', 'method',
       'data_set'],
      dtype='object')


...

Processing image 5: type(img)=<class 'numpy.ndarray'>, shape(img)=(1024, 1024, 3), type(mask)=<class 'torch.Tensor'>, shape(mask)=torch.Size([1024, 1024])
Processing image 5: type(img)=<class 'numpy.ndarray'>, shape(img)=(1024, 1024, 3), type(mask)=<class 'torch.Tensor'>, shape(mask)=torch.Size([1024, 1024])
Processing image 5: type(img)=<class 'numpy.ndarray'>, shape(img)=(1024, 1024, 3), type(mask)=<class 'torch.Tensor'>, shape(mask)=torch.Size([1024, 1024])
Processing image 5: type(img)=<class 'numpy.ndarray'>, shape(img)=(1024, 1024, 3), type(mask)=<class 'torch.Tensor'>, shape(mask)=torch.Size([1024, 1024])
Processing image 5: type(img)=<class 'numpy.ndarray'>, shape(img)=(1024, 1024, 3), type(mask)=<class 'torch.Tensor'>, shape(mask)=torch.Size([1024, 1024])
Processing image 5: type(img)=<class 'numpy.ndarray'>, shape(img)=(1024, 1024, 3), type(mask)=<class 'torch.Tensor'>, shape(mask)=torch.Size([1024, 1024])
Processing image 5: type(img)=<class 'numpy.ndarray'>, shape(img)=(102




Processing batch 6/66:   0%|          | 0/66 [57:51<?, ?it/s]

Processing image 6: type(img)=<class 'numpy.ndarray'>, shape(img)=(1024, 1024, 3), type(mask)=<class 'torch.Tensor'>, shape(mask)=torch.Size([1024, 1024])
Processing image 6: type(img)=<class 'numpy.ndarray'>, shape(img)=(1024, 1024, 3), type(mask)=<class 'torch.Tensor'>, shape(mask)=torch.Size([1024, 1024])
Processing image 6: type(img)=<class 'numpy.ndarray'>, shape(img)=(1024, 1024, 3), type(mask)=<class 'torch.Tensor'>, shape(mask)=torch.Size([1024, 1024])
Processing image 6: type(img)=<class 'numpy.ndarray'>, shape(img)=(1024, 1024, 3), type(mask)=<class 'torch.Tensor'>, shape(mask)=torch.Size([1024, 1024])
Processing image 6: type(img)=<class 'numpy.ndarray'>, shape(img)=(1024, 1024, 3), type(mask)=<class 'torch.Tensor'>, shape(mask)=torch.Size([1024, 1024])
Processing image 6: type(img)=<class 'numpy.ndarray'>, shape(img)=(1024, 1024, 3), type(mask)=<class 'torch.Tensor'>, shape(mask)=torch.Size([1024, 1024])
Processing image 6: type(img)=<class 'numpy.ndarray'>, shape(img)=(102




Processing batch 7/66:   0%|          | 0/66 [1:04:05<?, ?it/s]

Processing image 7: type(img)=<class 'numpy.ndarray'>, shape(img)=(1024, 1024, 3), type(mask)=<class 'torch.Tensor'>, shape(mask)=torch.Size([1024, 1024])
Processing image 7: type(img)=<class 'numpy.ndarray'>, shape(img)=(1024, 1024, 3), type(mask)=<class 'torch.Tensor'>, shape(mask)=torch.Size([1024, 1024])
Processing image 7: type(img)=<class 'numpy.ndarray'>, shape(img)=(1024, 1024, 3), type(mask)=<class 'torch.Tensor'>, shape(mask)=torch.Size([1024, 1024])
Processing image 7: type(img)=<class 'numpy.ndarray'>, shape(img)=(1024, 1024, 3), type(mask)=<class 'torch.Tensor'>, shape(mask)=torch.Size([1024, 1024])
Processing image 7: type(img)=<class 'numpy.ndarray'>, shape(img)=(1024, 1024, 3), type(mask)=<class 'torch.Tensor'>, shape(mask)=torch.Size([1024, 1024])
Processing image 7: type(img)=<class 'numpy.ndarray'>, shape(img)=(1024, 1024, 3), type(mask)=<class 'torch.Tensor'>, shape(mask)=torch.Size([1024, 1024])
Processing image 7: type(img)=<class 'numpy.ndarray'>, shape(img)=(102

In [4]:
import torch

if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"CUDA is available. Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device('cpu')
    print("CUDA is not available. Using CPU.")


CUDA is available. Using GPU: NVIDIA GeForce RTX 3050 4GB Laptop GPU
